<a href="https://colab.research.google.com/github/OneFineStarstuff/TheOneEverAfter/blob/main/_Tensor_Network_Expansions_(For_keep_updating).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import opt_einsum as oe
import torch


class PEPS:
    def __init__(self, Lx, Ly, d, D, use_gpu=False):
        self.Lx = Lx  # Number of rows
        self.Ly = Ly  # Number of columns
        self.d = d    # Physical dimension
        self.D = D    # Bond dimension
        self.use_gpu = use_gpu

        # Initialize tensors
        self.tensors = [
            [
                self.initialize_tensor(d, D)
                for _ in range(Ly)
            ]
            for _ in range(Lx)
        ]

    def initialize_tensor(self, d, D):
        """
        Initialize a PEPS tensor with shape (physical, left, right, up, down).
        """
        tensor = np.random.randn(d, D, D, D, D)
        if self.use_gpu:
            tensor = torch.tensor(tensor, dtype=torch.float32).to('cuda')
        else:
            tensor = torch.tensor(tensor, dtype=torch.float32)
        return tensor

    def apply_hamiltonian(self, h):
        """
        Apply a local Hamiltonian to each tensor.
        """
        print(f"Applying Hamiltonian: {h.shape}")
        for i in range(self.Lx):
            for j in range(self.Ly):
                tensor = self.tensors[i][j]
                self.tensors[i][j] = oe.contract("ab,bijkl->aijkl", h, tensor)

    def compute_observable(self, observable):
        """
        Compute an observable over the entire PEPS.
        """
        print(f"Computing observable: {observable.shape}")
        result = 0.0
        for row in self.tensors:
            for tensor in row:
                physical_contracted = oe.contract("ab,aijkl->bijkl", observable, tensor)
                result += physical_contracted.sum().item()
        return result

    def compute_multi_site_observable(self, observable, sites):
        """
        Compute a multi-site observable over specified sites.
        """
        print(f"Computing multi-site observable on sites: {sites}")
        contracted_tensors = []
        for (i, j) in sites:
            tensor = self.tensors[i][j]
            physical_contracted = oe.contract("ab,aijkl->bijkl", observable, tensor)
            contracted_tensors.append(physical_contracted)

        result = contracted_tensors[0]
        for idx, t in enumerate(contracted_tensors[1:]):
            print(f"Contracting tensor {idx+1}: result.shape={result.shape}, t.shape={t.shape}")
            result, t = self.ensure_bond_match(result, t)
            result = oe.contract("...ij,...jk->...ik", result, t)
        return result.sum().item()

    def full_contraction(self):
        """
        Perform a full contraction of the PEPS.
        """
        print("Performing full contraction...")
        row_contractions = []
        for row_idx, row in enumerate(self.tensors):
            print(f"Contracting row {row_idx}...")
            row_contracted = self.contract_row(row)
            print(f"Row {row_idx} contraction result: shape={row_contracted.shape}")
            row_contractions.append(row_contracted)

        result = row_contractions[0]
        for i in range(1, len(row_contractions)):
            print(
                f"Contracting rows: result.shape={result.shape}, "
                f"row_contractions[{i}].shape={row_contractions[i].shape}"
            )
            result, row_contractions[i] = self.ensure_bond_match(result, row_contractions[i])
            result = oe.contract("...ij,...jk->...ik", result, row_contractions[i])
        return result.sum().item()

    def contract_row(self, row_tensors):
        """
        Contract tensors in a single row along the horizontal bonds.
        """
        contracted = row_tensors[0]
        for idx, t in enumerate(row_tensors[1:]):
            print(
                f"Contracting row tensors: contracted.shape={contracted.shape}, "
                f"t[{idx+1}].shape={t.shape}"
            )
            contracted, t = self.ensure_bond_match(contracted, t)
            contracted = oe.contract("...lm,...mn->...ln", contracted, t)
        return contracted

    def ensure_bond_match(self, t1, t2):
        """
        Ensures that bond dimensions of two tensors are aligned via zero-padding.
        """
        def pad_tensor(t, target_shape):
            """
            Zero-pad a tensor to the target shape.
            """
            pad_sizes = [(0, max(0, target_dim - current_dim)) for current_dim, target_dim in zip(t.shape, target_shape)]
            pad_sizes = [item for sublist in reversed(pad_sizes) for item in sublist]
            return torch.nn.functional.pad(t, pad_sizes)

        # Print tensor shapes for debugging
        print(f"Ensuring bond match: t1.shape={t1.shape}, t2.shape={t2.shape}")

        # Pad all dimensions to the maximum shape
        max_shape = tuple(max(s1, s2) for s1, s2 in zip(t1.shape, t2.shape))
        t1_padded = pad_tensor(t1, max_shape)
        t2_padded = pad_tensor(t2, max_shape)

        print(f"After padding: t1_padded.shape={t1_padded.shape}, t2_padded.shape={t2_padded.shape}")
        return t1_padded, t2_padded


# Example Usage
if __name__ == "__main__":
    # Lattice dimensions
    Lx, Ly = 4, 4
    # Physical and bond dimensions
    d, D = 2, 3
    # Check for GPU
    use_gpu = torch.cuda.is_available()

    # Initialize PEPS
    peps = PEPS(Lx, Ly, d, D, use_gpu=use_gpu)

    # Initialize random Hamiltonian
    hamiltonian = np.random.randn(d, d)
    if use_gpu:
        hamiltonian = torch.tensor(hamiltonian, dtype=torch.float32).to('cuda')
    else:
        hamiltonian = torch.tensor(hamiltonian, dtype=torch.float32)

    # Apply Hamiltonian
    peps.apply_hamiltonian(hamiltonian)

    # Initialize random observable
    observable = np.random.randn(d, d)
    if use_gpu:
        observable = torch.tensor(observable, dtype=torch.float32).to('cuda')
    else:
        observable = torch.tensor(observable, dtype=torch.float32)

    # Compute single-site observable
    observable_result = peps.compute_observable(observable)
    print(f"Observable result: {observable_result}")

    # Compute multi-site observable
    multi_site_observable = np.random.randn(d, d)
    if use_gpu:
        multi_site_observable = torch.tensor(multi_site_observable, dtype=torch.float32).to('cuda')
    else:
        multi_site_observable = torch.tensor(multi_site_observable, dtype=torch.float32)

    multi_site_result = peps.compute_multi_site_observable(multi_site_observable, [(0, 0), (1, 1)])
    print(f"Multi-site observable result: {multi_site_result}")

    # Perform full contraction
    result = peps.full_contraction()
    print(f"Full contraction result: {result}")